# Nature Conservancy Fish Classification - Conv Model

### Imports & environment

In [1]:
import os

from keras.callbacks import ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator

from utils import * 
from vgg16bn import Vgg16BN

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/train'
%matplotlib inline

Using Theano backend.


### Config & Hyperparameters

In [2]:
# paths
data_path = DATA_HOME_DIR + '/' 
split_train_path = data_path + 'train/'
full_train_path = data_path + '/train_full/'
valid_path = data_path + '/valid/'
test_path = DATA_HOME_DIR + '/test/'
model_path = ROOT_DIR + '/models/vggbn_conv_640x360/'
submission_path = ROOT_DIR + '/submissions/vggbn_conv_640x360/'

# data
batch_size = 32
nb_split_train_samples = 10 #3327
nb_full_train_samples = 3777
nb_valid_samples = 450
nb_test_samples = 1000
classes = ["ALB", "BET", "DOL", "LAG", "NoF", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

# model
nb_aug = 5
n_filters = 158
lr = 1e-3
dropout = 0.5
clip = 0.01

In [3]:
def get_classes(trn_path, val_path, test_path):
    batches = get_batches(trn_path, shuffle=False, batch_size=1)
    val_batches = get_batches(val_path, shuffle=False, batch_size=1)
    test_batches = get_batches(test_path, shuffle=False, batch_size=1)
    return (val_batches.classes, batches.classes, onehot(val_batches.classes), onehot(batches.classes),
        val_batches.filenames, batches.filenames, test_batches.filenames)

In [4]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(split_train_path, valid_path, test_path)

Found 3398 images belonging to 8 classes.
Found 454 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


### Build Model and Precompute/Load Conv Features

In [5]:
vgg640 = Vgg16BN((360, 640)).model
vgg640.pop()
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

/Users/PeterWang/Documents/code/ds/lib/python2.7/site-packages/keras/layers/core.py:640: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 360, 640)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
vgg16bn.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
vgg16bn.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
vgg16bn.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
vgg16bn.py:34: UserWarning: Update your `Conv2D` call to t

In [10]:
batches = get_batches(split_train_path, batch_size=1, target_size=(360, 640), shuffle=False,
                      class_mode=None)

def new_batches():
    i = 0
    for x in batches:
        if i == 10:
            break
        print x.shape
        yield x.reshape(1, 3, 360, 640)
        i += 1

print 'batches updating.'
l_batches = new_batches()
print 'batches done.'
conv_trn_feat = vgg640.predict_generator(l_batches, nb_split_train_samples)
save_array(data_path + 'precomputed/trn_ft_640.dat', conv_trn_feat)

del conv_trn_feat

Found 3398 images belonging to 8 classes.
batches updating.
batches done.
(1, 360, 640, 3)
(1, 360, 640, 3)


ValueError: CorrMM: impossible output shape
  bottom shape: 1 x 640 x 5 x 362
  weights shape: 3 x 3 x 64 x 3
  top shape: 1 x 3 x -58 x 360

Apply node that caused the error: CorrMM{valid, (1, 1), (1, 1)}(InplaceDimShuffle{0,3,1,2}.0, Subtensor{::, ::, ::int64, ::int64}.0)
Toposort index: 63
Inputs types: [TensorType(float32, 4D), TensorType(float32, 4D)]
Inputs shapes: [(1, 640, 5, 362), (3, 3, 64, 3)]
Inputs strides: [(2560, 4, 926720, 2560), (4, 12, -108, -36)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[InplaceDimShuffle{0,2,3,1}(CorrMM{valid, (1, 1), (1, 1)}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "<ipython-input-5-a6794fdf9bae>", line 1, in <module>
    vgg640 = Vgg16BN((360, 640)).model
  File "vgg16bn.py", line 28, in __init__
    self.build()
  File "vgg16bn.py", line 47, in build
    self.ConvBlock(2, 64)
  File "vgg16bn.py", line 34, in ConvBlock
    model.add(Convolution2D(filters, 3, 3, activat

In [7]:
# val_batches = get_batches(valid_path, batch_size=1, target_size=(360, 640), shuffle=False,
#                           class_mode=None)
# conv_val_feat = vgg640.predict_generator(val_batches, nb_valid_samples)
# save_array(data_path + 'precomputed/val_ft_640.dat', conv_val_feat)

# del conv_val_feat

In [8]:
# test_batches = get_batches(test_path, batch_size=1, target_size=(360, 640), shuffle=False,
#                            class_mode=None)
# conv_test_feat = vgg640.predict_generator(test_batches, nb_test_samples)
# save_array(data_path+'precomputed/test_ft_640.dat', conv_test_feat)

# del conv_test_feat

In [9]:
conv_val_feat = load_array(data_path + 'precomputed/val_ft_640.dat')
conv_trn_feat = load_array(data_path + 'precomputed/trn_ft_640.dat')
conv_test_feat = load_array(data_path + 'precomputed/test_ft_640.dat')

### Train Model

In [10]:
conv_layers, _ = split_at(vgg640, Convolution2D)

In [11]:
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(n_filters, 3, 3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(n_filters, 3, 3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(n_filters, 3, 3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1, 2)),
        Convolution2D(8, 3, 3, border_mode='same'),
        Dropout(dropout),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

In [12]:
lrg_model = Sequential(get_lrg_layers())

lrg_model.summary()

/home/robert/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/robert/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/robert/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNormal(None, 512, 22, 40)   1024        batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 158, 22, 40)   728222      batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormal(None, 158, 22, 40)   316         convolution2d_14[0][0]           
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 158, 11, 20)   0           batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [13]:
lrg_model.compile(Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])

model_fn = model_path + '{val_loss:.2f}-loss_{epoch}epoch_640x360_vgg16bn.h5'
ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                       save_best_only=True, save_weights_only=True)

In [14]:
lrg_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=2, verbose=2,
             validation_data=(conv_val_feat, val_labels), callbacks=[ckpt])

Train on 3327 samples, validate on 450 samples
Epoch 1/2
17s - loss: 0.7446 - acc: 0.7719 - val_loss: 1.6365 - val_acc: 0.6067
Epoch 2/2
17s - loss: 0.1966 - acc: 0.9534 - val_loss: 0.4040 - val_acc: 0.8444


In [15]:
lrg_model.optimizer.lr /= 10

lrg_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=5, verbose=2,
             validation_data=(conv_val_feat, val_labels), callbacks=[ckpt])

Train on 3327 samples, validate on 450 samples
Epoch 1/5
17s - loss: 0.1241 - acc: 0.9684 - val_loss: 0.2563 - val_acc: 0.9089
Epoch 2/5
20s - loss: 0.0884 - acc: 0.9772 - val_loss: 0.1906 - val_acc: 0.9333
Epoch 3/5
20s - loss: 0.0628 - acc: 0.9838 - val_loss: 0.3516 - val_acc: 0.9156
Epoch 4/5
18s - loss: 0.0436 - acc: 0.9877 - val_loss: 0.1816 - val_acc: 0.9511
Epoch 5/5
16s - loss: 0.0279 - acc: 0.9919 - val_loss: 0.1982 - val_acc: 0.9489


In [16]:
lrg_model.optimizer.lr /= 10

lrg_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=5, verbose=2,
             validation_data=(conv_val_feat, val_labels), callbacks=[ckpt])

Train on 3327 samples, validate on 450 samples
Epoch 1/5
16s - loss: 0.0536 - acc: 0.9844 - val_loss: 0.2889 - val_acc: 0.9311
Epoch 2/5
16s - loss: 0.0746 - acc: 0.9787 - val_loss: 0.2635 - val_acc: 0.9178
Epoch 3/5
16s - loss: 0.0472 - acc: 0.9871 - val_loss: 0.1677 - val_acc: 0.9489
Epoch 4/5
16s - loss: 0.0166 - acc: 0.9943 - val_loss: 0.1362 - val_acc: 0.9644
Epoch 5/5
16s - loss: 0.0066 - acc: 0.9985 - val_loss: 0.1114 - val_acc: 0.9667


In [17]:
lrg_model.optimizer.lr /= 10

lrg_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=5, verbose=2,
             validation_data=(conv_val_feat, val_labels), callbacks=[ckpt])

Train on 3327 samples, validate on 450 samples
Epoch 1/5
16s - loss: 0.0069 - acc: 0.9982 - val_loss: 0.0850 - val_acc: 0.9733
Epoch 2/5
36s - loss: 0.0181 - acc: 0.9970 - val_loss: 0.0896 - val_acc: 0.9711
Epoch 3/5
22s - loss: 0.0050 - acc: 0.9991 - val_loss: 0.0937 - val_acc: 0.9711
Epoch 4/5
19s - loss: 0.0028 - acc: 0.9991 - val_loss: 0.1420 - val_acc: 0.9689
Epoch 5/5
17s - loss: 0.0016 - acc: 0.9997 - val_loss: 0.0806 - val_acc: 0.9756


In [23]:
def gen_preds_from_saved(use_all=True, weights_file=None):
    model = Sequential(get_lrg_layers())

    if use_all:
        preds = np.zeros((nb_test_samples, nb_classes))
        
        for root, dirs, files in os.walk(model_path):
            n_mods = 0
            for f in files:
                model.load_weights(model_path + f)
                preds += model.predict(conv_test_feat, batch_size=batch_size)
                n_mods += 1

        preds /= n_mods

    else:
        model.load_weights(model_path + weights_file)
        preds = model.predict(conv_test_feat, batch_size=batch_size)
        
    return preds
    

    
def gen_preds(model):
    
    if nb_aug:
        
        gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.05, zoom_range=0.05,
                                 channel_shift_range=10, height_shift_range=0.05, shear_range=0.05,
                                 horizontal_flip=True)
        predictions = np.zeros(shape=(nb_test_samples, nb_classes))
        
        for aug in range(nb_aug):
            
            test_batches = get_batches(test_path, batch_size=1, target_size=(360, 640), shuffle=False,
                                       class_mode=None, gen=gen)
            conv_test_feat = vgg640.predict_generator(test_batches, nb_test_samples)
            predictions += model.predict(conv_test_feat, batch_size=batch_size)
            
        predictions /= nb_aug
        
    else:
        predictions = model.predict(conv_test_feat, batch_size=batch_size)
        
    return predictions


# preds = gen_preds_from_saved(use_all=True, weights_file=None)
preds = gen_preds(lrg_model)

Found 1000 images belonging to 1 classes.
Found 1000 images belonging to 1 classes.
Found 1000 images belonging to 1 classes.
Found 1000 images belonging to 1 classes.
Found 1000 images belonging to 1 classes.


In [24]:
def write_submission(predictions, filenames):
    preds = np.clip(predictions, clip, 1-clip)
    sub_fn = submission_path + '{0}-aug_{1}clip_vgg_bn'.format(nb_aug, clip)

    with open(sub_fn + '.csv', 'w') as f:
        print("Writing Predictions to CSV...")
        f.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')
        for i, image_name in enumerate(filenames):
            pred = ['%.6f' % p for p in preds[i, :]]
            f.write('%s,%s\n' % (os.path.basename(image_name), ','.join(pred)))
        print("Done.")

write_submission(preds, test_filenames)

Writing Predictions to CSV...
Done.
